In [1]:
!pip install newspaper3k
!pip install pandas
!pip install matplotlib
!pip install transformers
!pip install nltk
!pip install sentencepiece
!pip install ipywidgets
# using nvidia-docker image from https://ngc.nvidia.com/catalog/containers/nvidia:tensorflow/tags
# using release 21.02-tf2-py3

     |████████████████████████████████| 211 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 7.9 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 7.4 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 80 kB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.4 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 115 kB 4.6 MB/s eta 0:00:01
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13537 sha256=b6168efd71821a197f6f0222379da44b0192509659eb60ab407ffbe012db5493
  Stored in directory: /root/.cache/pip/wheels/99/74/83/8fac1c8d9c648cfabebbbffe97a889f6624817f3aa0bbe6c09
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=f35e6943bb3d5a0cccd2a9f276bb5cdd5c511f3f7630ce6b99b3a82f2f3930c8
  Stored in directory: /root/.cache/pip/wheels/b6/09/68/a9f15498ac02c23dde29f18745bc6a6f574ba4ab41

In [2]:
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
from time import time
import io
import re

import pickle
from csv import reader
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras import layers
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten

from datetime import datetime
from collections import defaultdict




In [4]:
# from transformers import BertTokenizer, TFBertModel

In [5]:
from transformers import TFT5ForConditionalGeneration, T5Tokenizer

In [7]:
data = pd.read_table("biaseddata.csv", sep='\t', index_col=0)
data

,url,q3,perceived,primary.topic,secondary.topic,democrat.vote,republican.vote,article,length,democrat.bias,republican.bias,isbiased,biascategories
0,http://www.usatoday.com/story/news/nation/2013...,other,1,Civil Rights,Civil Rights,Neutral,Neutral,Maggie Clark Pew Stateline Staff Writer 21 s...,702,0.0,0.0,0,0
1,http://www.huffingtonpost.com/2013/05/21/senat...,News,0,Civil Rights,Civil Rights,Neutral,Neutral,WASHINGTON -- The Senate Judiciary Committee a...,1118,0.0,0.0,0,0
2,http://www.washingtonpost.com/opinions/dont-le...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral,David Cole is a professor of constitutional la...,1401,0.0,0.0,0,0
3,http://www.foxnews.com/politics/2013/04/03/oba...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral,"Parts of ObamaCare are starting to fray, even ...",798,0.0,0.0,0,0
4,http://www.breitbart.com/Big-Government/2013/1...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral,U.S. Immigration Citizenship and Immigration S...,947,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16584,http://www.washingtonpost.com/opinions/dana-mi...,News,1,Republican Scandals,Republican Scandals,SomewhatPositive,SomewhatNegative,President Obama won reelection in part by beat...,768,0.5,-0.5,1,1
16585,http://news.yahoo.com/republicans-hatred-obama...,Opinion,1,Republican Scandals,Republican Scandals,SomewhatPositive,SomewhatNegative,"Red-faced Republicans, circling and preparing ...",805,0.5,-0.5,1,1
16586,http://www.washingtonpost.com/opinions/dana-mi...,News,1,Republican Scandals,Republican Scandals,Neutral,Neutral,Rep. Steve Stockman’s moment as a viable Senat...,896,0.0,0.0,0,0
16587,http://www.washingtonpost.com/opinions/ej-dion...,News,1,Republican Scandals,Republican Scandals,SomewhatPositive,SomewhatNegative,We interrupt this highly partisan and ideologi...,751,0.5,-0.5,1,1


In [8]:
t5_model = 't5-small'

t5_tokenizer = T5Tokenizer.from_pretrained(t5_model)